# 🎙️ Voice Chatbot using Whisper + Cohere

### Install Required Packages

In [ ]:
# Install all necessary libraries (run only once)
# !pip install -r requirements.txt   # ← تأكد أنك شغلتها مرة على بيئتك فقط

### Import Libraries and Set API Key

In [ ]:
import tkinter as tk
import sounddevice as sd
from scipy.io.wavfile import write
import whisper
import cohere
from gtts import gTTS
from playsound import playsound
import os


# --------- Initialize Cohere Client ---------
co = cohere.Client("YOUR_API_KEY")  # Replace with your actual Cohere API Key

### Load Whisper Model

In [2]:
# Load Whisper speech-to-text model (base size)
whisper_model = whisper.load_model("base")


###  Define Voice-to-Response Function

In [3]:
def record_and_process():
    # Update status to indicate recording is in progress
    status_label.config(text="🎙️ Recording...") 
    window.update()

    # Record audio from the microphone for 5 seconds
    duration = 5  
    fs = 44100  # Sampling rate
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait until recording is finished

    # Save the recording to a WAV file
    audio_path = "recorded.wav"
    write(audio_path, fs, recording)

    # Update status to indicate transcription is in progress
    status_label.config(text="📝 Transcribing...")
    window.update()

    # Use Whisper to transcribe the audio
    result = whisper_model.transcribe(audio_path)
    transcribed_text = result["text"]
    transcribed_label.config(text=f"You said: {transcribed_text}")

    # Update status to indicate response generation is in progress
    status_label.config(text="⏳ Generating response...")
    window.update()

    # Send transcribed text to Cohere to generate a response
    response = co.chat(
        model="command-r",
        message=transcribed_text
    )
    response_text = response.text
    response_label.config(text=f"🤖 {response_text}")

    # Update status to indicate text-to-speech is in progress
    status_label.config(text="🔊 Playing response...")
    window.update()

    # Convert the response to speech and play it
    tts = gTTS(text=response_text, lang='en')
    output_path = "response.mp3"
    tts.save(output_path)
    playsound(output_path)

    # Clean up temporary audio files
    os.remove(audio_path)
    os.remove(output_path)

    # Update status to indicate the app is ready again
    status_label.config(text="✅ Ready to record again.")


### Create and Run GUI

In [5]:
# --------- GUI Interface ---------
window = tk.Tk()
window.title("🤖 Voice Chatbot ")
window.geometry("500x350")  # Bigger window

# Record button - blue background, white text
record_btn = tk.Button(window, text="🎙️ Start Recording", font=("Arial", 16, "bold"),
                       bg="#89AFD7", fg="white", activebackground="#5d95d1", activeforeground="white",
                       command=record_and_process)
record_btn.pack(pady=(20, 10), ipadx=10, ipady=5)

# Label to show transcribed text (user speech)
transcribed_label = tk.Label(window, text="", font=("Arial", 14), fg="#0D0D0D", wraplength=460, justify="left")
transcribed_label.pack(pady=(0, 10))

# Label to show chatbot response text with emoji
response_label = tk.Label(window, text="", font=("Arial", 14, "italic"), fg="#007BFF", wraplength=460, justify="left")
response_label.pack(pady=(0, 30))

# Status label to show current processing status (small font, grey)
status_label = tk.Label(window, text="Ready to record.", font=("Arial", 10), fg="gray")
status_label.pack()

# Quit button - red background, white text
quit_btn = tk.Button(window, text="❌ Quit", font=("Arial", 14, "bold"),
                     bg="#d25662", fg="white", activebackground="#81373e", activeforeground="white",
                     command=window.destroy)
quit_btn.pack(side="bottom", pady=15, ipadx=10, ipady=5)

window.mainloop()

C:\Users\iphon\AppData\Roaming\Python\Python310\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
